In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

file_train="../data/raw/train.csv"
file_test="../data/raw/test.csv"


data=pd.read_csv(file_train,delimiter=",")

y_train=data["Survived"]
x_train=data.drop(columns=["Survived"])

print(x_train)

     PassengerId  Pclass                                               Name  \
0              1       3                            Braund, Mr. Owen Harris   
1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2              3       3                             Heikkinen, Miss. Laina   
3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4              5       3                           Allen, Mr. William Henry   
..           ...     ...                                                ...   
886          887       2                              Montvila, Rev. Juozas   
887          888       1                       Graham, Miss. Margaret Edith   
888          889       3           Johnston, Miss. Catherine Helen "Carrie"   
889          890       1                              Behr, Mr. Karl Howell   
890          891       3                                Dooley, Mr. Patrick   

        Sex   Age  SibSp  Parch            Ticket  

In [7]:
#Filling missing values of age and Embarked

x_train["Age"]=x_train.groupby(["Pclass","Sex"])["Age"].transform(lambda x:x.fillna(x.median()))

x_train["Embarked"]=x_train["Embarked"].ffill()


In [8]:
#Feature Engineering
x_train["FamilySize"]=x_train["SibSp"]+x_train["Parch"]+1

x_train["isAlone"]=x_train["FamilySize"]==1

#Extracting Titles from names
x_train["Title"]=x_train["Name"].str.extract(r",\s*([^\.]+)\.")
x_train["Title"]=x_train["Title"].replace(["Mlle","Ms"],"Miss")
x_train["Title"]=x_train["Title"].replace(["Mme"],"Mrs")
#Extracting ticket Class or group
x_train["TicketPrefix"]=x_train["Ticket"].str.extract(r"^([A-Za-z./]+)")
x_train["TicketPrefix"]=x_train["TicketPrefix"].fillna("Unknown")

#Extracting Cabin Floor
x_train["CabinLetter"]=x_train["Cabin"].str[0]
x_train["CabinLetter"]=x_train["CabinLetter"].fillna("Unknown")


x_train=x_train.drop(columns=["SibSp","Parch","Name","Ticket","Cabin"])


In [9]:
#One Hot encoding
x_train=pd.get_dummies(x_train,columns=["Pclass","Sex","Embarked","Title","TicketPrefix","CabinLetter"])
print(x_train)

     PassengerId   Age     Fare  FamilySize  isAlone  Pclass_1  Pclass_2  \
0              1  22.0   7.2500           2    False     False     False   
1              2  38.0  71.2833           2    False      True     False   
2              3  26.0   7.9250           1     True     False     False   
3              4  35.0  53.1000           2    False      True     False   
4              5  35.0   8.0500           1     True     False     False   
..           ...   ...      ...         ...      ...       ...       ...   
886          887  27.0  13.0000           1     True     False      True   
887          888  19.0  30.0000           1     True      True     False   
888          889  21.5  23.4500           4    False     False     False   
889          890  26.0  30.0000           1     True      True     False   
890          891  32.0   7.7500           1     True     False     False   

     Pclass_3  Sex_female  Sex_male  ...  TicketPrefix_WE/P  CabinLetter_A  \
0        

In [12]:
#Storing the Processed Data
filePath_train="../data/processed/train.csv"
filePath_test="../data/processed/test.csv"

filePath_train_output="../data/processed/train_output.csv"

y_train.to_csv(filePath_train_output,index=False)
x_train.to_csv(filePath_train,index=False)